In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error
from keras import losses


def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df


# 1. Carica il test set
test_set = pd.read_csv('../archive/test_set3_processed.csv')
test_set = reduce_memory_usage(test_set)
# Stampare il numero di righe e colonne
print(test_set.shape)

from keras import models
autoencoder=models.load_model('../archive/model3')
autoencoder.summary()

# 2. Separare le features e il target 
X_test = test_set.drop(columns=['target'])  # Rimuovi la colonna 'target' dalle features
y_true = test_set['target']  # Salva i veri target

samples_b = 2402405
n_features = 43
# normal event

#real_x_b trasforma il dataframe pandas in un array numpy 
real_x = X_test.to_numpy().reshape(samples_b, n_features)
# 4. Fai delle predizioni sul test set
predictions = autoencoder.predict(x=real_x)

# 5. Calcola l'errore di ricostruzione (MSE) per il test set
# mse_test = np.mean(np.square(X_test - predictions), axis=1)
mse_test=losses.mean_squared_error(real_x, predictions).numpy()
mse_test_df=pd.DataFrame({'mse': mse_test})
# 6. Applica la soglia per determinare le anomalie
threshold = 0.010797960832714944

y_pred = (mse_test_df.mse > threshold).astype(int)  # 1 per anomalia, 0 per normale

import numpy as np

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Stampare i primi 10 valori di y_true e y_pred
print("Primi 10 valori di y_true:", y_true[:10])
print("Primi 10 valori di y_pred:", y_pred[:10])


print("Dimensioni di y_true:", y_true.shape)
print("Dimensioni di y_pred:", y_pred.shape)

# 7. Calcola le metriche di performance
accuracy = accuracy_score(y_true, y_pred)

# Precision e Recall separati per ciascuna classe (0=benigno, 1=maligno)
precision = precision_score(y_true, y_pred, average=None)
recall = recall_score(y_true, y_pred, average=None)

# 8. Matrice di confusione
conf_matrix = confusion_matrix(y_true, y_pred)

# 9. Stampa i risultati
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision Benigni (0): {precision[0]:.4f}")
print(f"Precision Maligni (1): {precision[1]:.4f}")
print(f"Recall Benigni (0): {recall[0]:.4f}")
print(f"Recall Maligni (1): {recall[1]:.4f}")
print("Matrice di Confusione:")
print(conf_matrix)


(2402405, 44)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Sequential)        (None, 5)                 1145      
                                                                 
 decoder (Sequential)        (None, 43)                1183      
                                                                 
Total params: 2,328
Trainable params: 2,328
Non-trainable params: 0
_________________________________________________________________
75076/75076 [==============================] - 156s 2ms/step
Primi 10 valori di y_true: [0 0 0 0 0 0 0 0 0 0]
Primi 10 valori di y_pred: [0 0 0 0 0 0 0 0 0 0]
Dimensioni di y_true: (2402405,)
Dimensioni di y_pred: (2402405,)
Accuracy: 0.8919
Precision Benigni (0): 0.8977
Precision Maligni (1): 0.7117
Recall Benigni (0): 0.9899
Recall Maligni (1): 0.1811
Matrice di Confusione:
[[2090049   21352]
 [ 238306   52698]]
